# MNIST project

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import datetime
import io
import itertools

import numpy as np
import sklearn.metrics

import matplotlib.pyplot as plt

# downlaoding and prepocessing the data

In [2]:
BUFFER_SIZE = 70_000
BATCH_SIZE = 128
NUM_EPOCHS = 20

In [3]:
mnist_dataset , mnist_info = tfds.load(name='mnist', with_info=True , as_supervised=True)
mnist_train , mnist_test = mnist_dataset['train'],mnist_dataset['test']

In [4]:
def scale(image,label):
    image = tf.cast(image, tf.float32) #casts the type to float32 to avoid errors
    image /= 255.
    return image , label

train_and_validation_data=mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [5]:
num_validation_samples = 0.1* mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64)

num_test_samples = 0.1* mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

In [6]:
train_and_validation_data = train_and_validation_data.shuffle(BUFFER_SIZE)

train_data = train_and_validation_data.skip(num_validation_samples) 
validation_data = train_and_validation_data.take(num_validation_samples)

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples) 
test_data = test_data.batch(num_test_samples)

In [7]:
for images, labels in validation_data:
    images_val = images.numpy()
    labels_val = labels.numpy()

# Creating , training and testing the model

- creating the model

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(50, 5, activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)), 
    tf.keras.layers.Conv2D(50, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)), 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

- creating an early stopping callback

In [11]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',    
    min_delta = 0,
    patience = 2,
    verbose = 0, 
    restore_best_weights = True
)

- preparing tensorboard

In [12]:
log_dir = "Logs\\fit\\" + "run-1"

log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

- preparing the confusion matrix

In [13]:
def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(12, 12))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    return figure

In [14]:
def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    
    # Closing the figure prevents it from being displayed directly inside the notebook.
    plt.close(figure)
    
    buf.seek(0)
    
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    
    return image

In [15]:
# Define a file writer variable for logging purposes
file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(images_val)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)
    
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

- creating the confusion matrix callback

In [16]:
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)

- training the model

In [17]:
model.fit(
    train_data, 
    epochs = NUM_EPOCHS, 
    callbacks = [tensorboard_callback , early_stopping], 
    validation_data = validation_data,
    verbose = 2
)

Epoch 1/20
422/422 - 31s - loss: 0.2802 - accuracy: 0.9215 - val_loss: 0.0857 - val_accuracy: 0.9733 - 31s/epoch - 73ms/step
Epoch 2/20
422/422 - 27s - loss: 0.0719 - accuracy: 0.9782 - val_loss: 0.0523 - val_accuracy: 0.9850 - 27s/epoch - 63ms/step
Epoch 3/20
422/422 - 28s - loss: 0.0540 - accuracy: 0.9836 - val_loss: 0.0465 - val_accuracy: 0.9852 - 28s/epoch - 66ms/step
Epoch 4/20
422/422 - 28s - loss: 0.0425 - accuracy: 0.9866 - val_loss: 0.0440 - val_accuracy: 0.9857 - 28s/epoch - 66ms/step
Epoch 5/20
422/422 - 25s - loss: 0.0369 - accuracy: 0.9887 - val_loss: 0.0351 - val_accuracy: 0.9903 - 25s/epoch - 59ms/step
Epoch 6/20
422/422 - 17s - loss: 0.0322 - accuracy: 0.9903 - val_loss: 0.0257 - val_accuracy: 0.9925 - 17s/epoch - 41ms/step
Epoch 7/20
422/422 - 17s - loss: 0.0274 - accuracy: 0.9913 - val_loss: 0.0271 - val_accuracy: 0.9923 - 17s/epoch - 41ms/step
Epoch 8/20
422/422 - 17s - loss: 0.0250 - accuracy: 0.9921 - val_loss: 0.0196 - val_accuracy: 0.9950 - 17s/epoch - 41ms/step


- testing the model

In [18]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

10/10 [==============================] - 1s 78ms/step - loss: 0.0341 - accuracy: 0.9917
Test loss: 0.0341. Test accuracy: 99.17%


- visualizing tensor board

In [20]:
%load_ext tensorboard
%tensorboard --logdir "logs/fit"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
